# **DATATÓN ACADÉMICO**

## **Reconocimiento Facial para registro de asistencia**

**Estudiante:** Yuneidy Lorena Gutierrez Diaz

**Espacio Académico:** Redes Neuronales


### **Objetivo General**

Diseñar mejoras al pipeline de reconocimiento facial implementado para registrar la asistencia de estudiantes. El reto
se enfoca en aumentar la precisión del modelo y mejorar su rendimiento bajo condiciones reales, sin requerir interfaz
ni despliegue.

## **Contexto Técnico**
Actualmente se utiliza un modelo funcional basado en detección de rostros con MTCNN, generación de embeddings
faciales con FaceNet, y comparación mediante distancias para determinar la identidad de los estudiantes. Este
sistema ha sido útil, pero requiere mejoras para lograr mayor robustez y precisión.


In [ ]:
!pip install facenet-pytorch torch torchvision numpy pillow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importar librerías necesarias
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import numpy as np
import pandas as pd
import requests
from io import BytesIO
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# --- Configuración FaceNet ---
# Configurar dispositivo (GPU si está disponible)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Inicializar detector y modelo de embeddings
mtcnn = MTCNN(image_size=160, margin=20, min_face_size=20, device=device)
facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)


In [ ]:
# --- Funciones comunes para cargar imágenes desde enlaces de Google Drive ---
def extraer_id_enlace(enlace):
    if "id=" in enlace:
        return enlace.split("id=")[-1].split("&")[0]
    elif "file/d/" in enlace:
        return enlace.split("file/d/")[-1].split("/")[0]
    else:
        return None

def cargar_imagen_desde_drive(file_id):
    url = f"https://drive.google.com/uc?id={file_id}"
    response = requests.get(url, stream=True)
    response.raise_for_status()
    imagen = Image.open(BytesIO(response.content)).convert('RGB')
    return imagen

In [ ]:
# --- Funciones de extracción de embeddings ---
def get_facenet_embedding_from_link(link):
    file_id = extraer_id_enlace(link)
    if file_id is None:
        return None
    try:
        img = cargar_imagen_desde_drive(file_id)
        face_tensor = mtcnn(img)
        if face_tensor is not None:
            face_tensor = face_tensor.unsqueeze(0).to(device)
            with torch.no_grad():
                emb = facenet(face_tensor)
            return emb.cpu().numpy().flatten()
    except Exception as e:
        print(f"Error FaceNet con {link}: {e}")
    return None

In [ ]:
# --- Funciones para FaceNet ---
def get_facenet_embedding_from_link(link):
    file_id = extraer_id_enlace(link)
    if file_id is None:
        return None
    try:
        img = cargar_imagen_desde_drive(file_id)
        face_tensor = mtcnn(img)
        if face_tensor is not None:
            face_tensor = face_tensor.unsqueeze(0).to(device)
            with torch.no_grad():
                emb = facenet(face_tensor)
            return emb.cpu().numpy().flatten()
    except Exception as e:
        print(f"Error FaceNet con {link}: {e}")
    return None

In [ ]:
# Leer archivo Excel con nombres y enlaces
excel_path = '/content/drive/MyDrive/Fotos RN (respuestas).xlsx'  # Ajusta la ruta
df = pd.read_excel(excel_path)

# Crear diccionario {nombre: [lista_de_enlaces]}
image_dict = {}
for idx, row in df.iterrows():
    nombre = str(row['Nombre Completo']).strip()
    fotos = str(row['Ingrese sus fotos']).split(',')
    fotos = [f.strip() for f in fotos if f.strip() != '']
    if nombre not in image_dict:
        image_dict[nombre] = []
    image_dict[nombre].extend(fotos)

# División train/test 70/30
def split_image_dict(image_dict, test_size=0.3, random_state=42):
    train_dict, test_dict = {}, {}
    for name, links in image_dict.items():
        if len(links) < 2:
            continue  # mínimo 2 imágenes para dividir
        train_links, test_links = train_test_split(links, test_size=test_size, random_state=random_state)
        train_dict[name] = train_links
        test_dict[name] = test_links
    return train_dict, test_dict

train_dict, test_dict = split_image_dict(image_dict)

In [ ]:
# Construir base de embeddings
def build_embeddings_db_from_links(image_dict, embedding_fn):
    db = {}
    for name, links in image_dict.items():
        embeddings = []
        for link in links:
            emb = embedding_fn(link)
            if emb is not None:
                embeddings.append(emb)
        if embeddings:
            db[name] = embeddings
    return db

print("Extrayendo embeddings FaceNet")
facenet_train_db = build_embeddings_db_from_links(train_dict, get_facenet_embedding_from_link)
facenet_test_db = build_embeddings_db_from_links(test_dict, get_facenet_embedding_from_link)


Extrayendo embeddings FaceNet


In [ ]:
# Evaluación con métricas
def recognize_and_evaluate(test_db, train_db):
    y_true, y_pred = [], []
    centroids = {name: np.mean(embs, axis=0) for name, embs in train_db.items()}
    for true_name, test_embs in test_db.items():
        for emb in test_embs:
            sims = {name: cosine_similarity([emb], [centroid])[0][0] for name, centroid in centroids.items()}
            pred_name = max(sims, key=sims.get)
            y_true.append(true_name)
            y_pred.append(pred_name)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    return acc, prec, rec

print("Evaluando FaceNet...")
acc, prec, rec = recognize_and_evaluate(facenet_test_db, facenet_train_db)
print(f"FaceNet - Accuracy: {acc:.4f}, Precisión: {prec:.4f}, Recall: {rec:.4f}")

Evaluando FaceNet...
FaceNet - Accuracy: 0.9000, Precisión: 0.8750, Recall: 0.9000


### **Interpretación de Resultados - FaceNet**
* Accuracy: 0.9000

FaceNet identificó correctamente el 90.00% de los rostros en el conjunto de prueba. Esto sugiere que FaceNet tiene una  capacidad para reconocer rostros en general en este conjunto de datos específico.

* Precisión: 0.8750

De todas las identificaciones que FaceNet hizo como positivas, el 87.50% fueron realmente correctas. Esto implica que FaceNet tiene una tasa de falsos positivos, lo que puede ser valioso en aplicaciones donde la precisión es crítica.

* Recall: 0.9000

FaceNet fue capaz de identificar correctamente el 90.00% de todos los rostros que realmente correspondían a una persona. Esto sugiere que FaceNet tiene una mayor capacidad para encontrar la mayoría de los rostros relevantes, minimizando aún más los falsos negativos.

Finalmente FaceNet muestra un rendimiento ligeramente superior en términos de exactitud y exhaustividad. Tiene una buena capacidad para reconocer rostros en general, con una tasa de falsos positivos muy baja y una alta capacidad para encontrar la mayoría de los rostros relevantes.

 ## **Explorar nuevos modelos o embeddings (como ArcFace o InsightFace).**

In [ ]:
!pip install onnxruntime insightface

In [ ]:
# Librerias necesarias
import numpy as np
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics.pairwise import cosine_similarity
import insightface
from insightface.app import FaceAnalysis

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.7'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
# Inicializar modelo ArcFace preentrenado
arcface_app = FaceAnalysis(name='buffalo_l')
arcface_app.prepare(ctx_id=0)  # Usa ctx_id=-1 si no tienes GPU

/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [ ]:
# Función para extraer embedding ArcFace desde enlace
def get_arcface_embedding_from_link(link):
    file_id = extraer_id_enlace(link)
    if file_id is None:
        return None
    try:
        img = np.array(cargar_imagen_desde_drive(file_id))
        faces = arcface_app.get(img)
        if len(faces) > 0:
            return faces[0].embedding
    except Exception as e:
        print(f"Error con {link}: {e}")
    return None

In [ ]:
# División 70/30 train/test
def split_image_dict(image_dict, test_size=0.3, random_state=42):
    train_dict, test_dict = {}, {}
    for name, links in image_dict.items():
        if len(links) < 2:
            continue  # Necesitas al menos 2 imágenes para dividir
        train_links, test_links = train_test_split(links, test_size=test_size, random_state=random_state)
        train_dict[name] = train_links
        test_dict[name] = test_links
    return train_dict, test_dict

train_dict, test_dict = split_image_dict(image_dict)

# Construir base de embeddings
def build_embeddings_db_from_links(image_dict, embedding_fn):
    db = {}
    for name, links in image_dict.items():
        embeddings = []
        for link in links:
            emb = embedding_fn(link)
            if emb is not None:
                embeddings.append(emb)
        if embeddings:
            db[name] = embeddings
    return db

print("Extrayendo embeddings ArcFace")
arcface_train_db = build_embeddings_db_from_links(train_dict, get_arcface_embedding_from_link)
arcface_test_db = build_embeddings_db_from_links(test_dict, get_arcface_embedding_from_link)


Extrayendo embeddings ArcFace


/usr/local/lib/python3.11/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [ ]:
#Evaluación con métricas
def recognize_and_evaluate(test_db, train_db):
    y_true, y_pred = [], []
    centroids = {name: np.mean(embs, axis=0) for name, embs in train_db.items()}
    for true_name, test_embs in test_db.items():
        for emb in test_embs:
            sims = {name: cosine_similarity([emb], [centroid])[0][0] for name, centroid in centroids.items()}
            pred_name = max(sims, key=sims.get)
            y_true.append(true_name)
            y_pred.append(pred_name)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    return acc, prec, rec

acc, prec, rec = recognize_and_evaluate(arcface_test_db, arcface_train_db)
print(f"ArcFace - Accuracy: {acc:.4f}, Precisión: {prec:.4f}, Recall: {rec:.4f}")


ArcFace - Accuracy: 0.8810, Precisión: 0.8690, Recall: 0.8810


### **Interpretación de Resultados - ArcFace**

* Accuracy: 0.8810

Indica que ArcFace identificó correctamente el 88.10% de los rostros en el conjunto de prueba. Esto es un buen resultado y sugiere que el modelo tiene una buena capacidad para reconocer rostros en general.

* Precisión: 0.8690

De todas las identificaciones que ArcFace hizo como positivas, el 86.90% fueron realmente correctas. Esto implica que el modelo tiene una baja tasa de falsos positivos, lo cual es importante en aplicaciones donde es crucial evitar identificaciones incorrectas.

* Recall: 0.8810

ArcFace fue capaz de identificar correctamente el 88.10% de todos los rostros que realmente correspondían a una persona. Esto sugiere que el modelo tiene una buena capacidad para encontrar la mayoría de los rostros relevantes, minimizando los falsos negativos.

Finalmente ArcFace muestra un rendimiento equilibrado con una buena exactitud, precisión y exhaustividad. Esto indica que es un modelo confiable para el reconocimiento facial, con una baja tasa de errores tanto en identificaciones incorrectas como en rostros no detectados. Este algoritmo utiliza redes convolucionales para generar vectores altamente discriminativos.

## Comparación de modelos



In [ ]:
# 1. Extraer embeddings y evaluar FaceNet
facenet_train_db = build_embeddings_db_from_links(train_dict, get_facenet_embedding_from_link)
facenet_test_db = build_embeddings_db_from_links(test_dict, get_facenet_embedding_from_link)
acc_fn, prec_fn, rec_fn,= recognize_and_evaluate(facenet_test_db, facenet_train_db)

In [ ]:
def recognize_and_evaluate(test_db, train_db):
    y_true, y_pred = [], []
    centroids = {name: np.mean(embs, axis=0) for name, embs in train_db.items()}
    for true_name, test_embs in test_db.items():
        for emb in test_embs:
            sims = {name: cosine_similarity([emb], [cent])[0][0] for name, cent in centroids.items()}
            pred_name = max(sims, key=sims.get)
            y_true.append(true_name)
            y_pred.append(pred_name)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    return acc, prec, rec, y_true, y_pred

In [ ]:
# 2. Extraer embeddings y evaluar ArcFace
arcface_train_db = build_embeddings_db_from_links(train_dict, get_arcface_embedding_from_link)
arcface_test_db = build_embeddings_db_from_links(test_dict, get_arcface_embedding_from_link)
acc_af, prec_af, rec_af, _, _ = recognize_and_evaluate(arcface_test_db, arcface_train_db)

# 3. Crear tabla comparativa automáticamente
import pandas as pd

df_comparacion = pd.DataFrame({
    'Modelo': ['FaceNet', 'ArcFace'],
    'Accuracy': [acc_fn, acc_af],
    'Precisión': [prec_fn, prec_af],
    'Recall': [rec_fn, rec_af]
})

print(df_comparacion)


/usr/local/lib/python3.11/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


    Modelo  Accuracy  Precisión    Recall
0  FaceNet  0.900000   0.875000  0.900000
1  ArcFace  0.880952   0.869048  0.880952


## **Evaluacion de la predicción de cada modelo**

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

def evaluar_predicciones(y_true, y_pred, nombre_modelo="Modelo"):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    print(f"Evaluación de {nombre_modelo}:")
    print(f"  Accuracy:  {acc:.4f}")
    print(f"  Precisión: {prec:.4f}")
    print(f"  Recall:    {rec:.4f}")
    print("\nReporte detallado por clase:")
    print(classification_report(y_true, y_pred, zero_division=0))
    print("-" * 40)


In [ ]:
acc_fn, prec_fn, rec_fn, y_true_fn, y_pred_fn = recognize_and_evaluate(facenet_test_db, facenet_train_db)
acc_af, prec_af, rec_af, y_true_af, y_pred_af = recognize_and_evaluate(arcface_test_db, arcface_train_db)

FaceNet

In [ ]:
evaluar_predicciones(y_true_fn, y_pred_fn, nombre_modelo="FaceNet")

Evaluación de FaceNet:
  Accuracy:  0.9000
  Precisión: 0.8750
  Recall:    0.9000

Reporte detallado por clase:
                                  precision    recall  f1-score   support

        Angela Rocio Rico Ortega       1.00      1.00      1.00         2
  Angela Tatiana orjuela guevara       1.00      1.00      1.00         2
    Camilo Andrés Castillo Riaño       1.00      1.00      1.00         2
        Carlos Esteban Diaz Ruan       1.00      1.00      1.00         2
     Diego Arturo Castro Beltran       1.00      1.00      1.00         2
            Felipe Herrera Riaño       0.33      0.50      0.40         2
  Hayder arley Rodriguez Orjuela       1.00      1.00      1.00         2
       JUAN FELIPE HERRERA RIANO       0.00      0.00      0.00         2
        Juan Camilo Ortiz Ibañez       1.00      1.00      1.00         2
                   Julian Mendez       1.00      1.00      1.00         1
     Kewin Damián gacha guayacan       0.67      1.00      0.80         

ArceFace

In [ ]:
evaluar_predicciones(y_true_af, y_pred_af, nombre_modelo="ArcFace")

Evaluación de ArcFace:
  Accuracy:  0.8810
  Precisión: 0.8690
  Recall:    0.8810

Reporte detallado por clase:
                                  precision    recall  f1-score   support

        Angela Rocio Rico Ortega       1.00      1.00      1.00         2
  Angela Tatiana orjuela guevara       1.00      1.00      1.00         2
    Camilo Andrés Castillo Riaño       1.00      1.00      1.00         1
        Carlos Esteban Diaz Ruan       1.00      1.00      1.00         2
     Diego Arturo Castro Beltran       1.00      1.00      1.00         2
            Felipe Herrera Riaño       0.33      0.50      0.40         2
  Hayder arley Rodriguez Orjuela       1.00      1.00      1.00         2
       JUAN FELIPE HERRERA RIANO       0.00      0.00      0.00         2
        Juan Camilo Ortiz Ibañez       1.00      1.00      1.00         2
                   Julian Mendez       1.00      1.00      1.00         1
     Kewin Damián gacha guayacan       1.00      0.67      0.80         

### **Comparación - Conclusión**

FaceNet muestra un rendimiento ligeramente superior a ArcFace en el conjunto de datos, pero las diferencias son pequeñas. Ambos modelos parecen ser adecuados para la tarea de reconocimiento facial, y la elección final podría depender de otros factores como la velocidad de inferencia, la facilidad de implementación y los requisitos específicos del caso de uso.

Estos modelos utilizan biometría facial con deep learning, pero no se trata la red neuronal toda la imagen, solo se usa como entrada el área seleccionada durante la detección. Además, para mejorar la precisión de la verificación facial, se puede realizar un alineamiento facial

# Predicciones con nueva imagen

In [ ]:
from PIL import Image

def get_facenet_embedding_local(img_path):
    img = Image.open(img_path).convert('RGB')
    face_tensor = mtcnn(img)
    if face_tensor is not None:
        face_tensor = face_tensor.unsqueeze(0).to(device)
        with torch.no_grad():
            emb = facenet(face_tensor)
        return emb.cpu().numpy().flatten()
    return None

In [ ]:
def get_arcface_embedding_from_link(link):
    file_id = extraer_id_enlace(link)
    if file_id is None:
        return None
    try:
        img = np.array(cargar_imagen_desde_drive(file_id))
        faces = arcface_app.get(img)
        if len(faces) > 0:
            return faces[0].embedding
    except Exception as e:
        print(f"Error ArcFace con {link}: {e}")
    return None

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def predict_identity(image_input, embedding_fn, train_db, threshold=0.5):
    emb = embedding_fn(image_input)
    if emb is None:
        return "No se pudo extraer embedding"
    centroids = {name: np.mean(embs, axis=0) for name, embs in train_db.items()}
    sims = {name: cosine_similarity([emb], [centroid])[0][0] for name, centroid in centroids.items()}
    best_match = max(sims, key=sims.get)
    best_score = sims[best_match]
    if best_score >= threshold:
        return best_match
    else:
        return "Desconocido"

In [ ]:
ruta_imagen = "/content/drive/MyDrive/1748222062989.jpg"  # Cambia por la ruta de tu imagen
prediccion = predict_identity(ruta_imagen, get_facenet_embedding_local, facenet_train_db, threshold=0.5)
print(f"Predicción FaceNet: {prediccion}")

Predicción FaceNet: Yuneidy Lorena Gutierrez Diaz


In [ ]:
enlace_imagen = 'https://drive.google.com/file/d/1dLuaxe6Q9jepAJZY7lpFS8YGZYU294AD/view?usp=sharing'  # Cambia este enlace# Cambia por tu enlace
prediccion = predict_identity(enlace_imagen, get_arcface_embedding_from_link, arcface_train_db, threshold=0.5)
print(f"Predicción ArcFace: {prediccion}")

/usr/local/lib/python3.11/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


Predicción ArcFace: JUAN FELIPE HERRERA RIANO


### **R5: Reflexión Final – Limitaciones, Errores Frecuentes y Aspectos Éticos**

* **Limitaciones Técnicas:**

Cantidad de imágenes por persona: Algunas clases tienen solo 2 o 3 fotos, lo que limita la generalización del modelo.

Condiciones variables: Cambios en iluminación, expresión o ángulo afectan la calidad del embedding.

Dependencia de internet: La descarga de imágenes desde Drive introduce latencias y posibles errores si el enlace está roto.

* **Errores Frecuentes:**

Falsos positivos: Algunos estudiantes pueden ser mal identificados si sus rostros son similares o la imagen está mal tomada.

Embeddings nulos: Algunas imágenes no permiten detectar rostros (por baja calidad, mal encuadre o error de red).

Overfitting con pocas imágenes: Al tener una sola imagen por persona en entrenamiento, el modelo puede ajustarse demasiado a esa pose o condición específica.

* **Aspectos Éticos:**

Privacidad y consentimiento: Es fundamental que los estudiantes hayan dado consentimiento informado para que sus imágenes sean usadas con fines académicos.

Seguridad de datos sensibles: Al tratar con imágenes faciales, se deben tomar medidas para proteger los datos y evitar usos indebidos.

Sesgo algorítmico: Si hay sesgo en la recolección de imágenes (por ejemplo, más imágenes de ciertos grupos demográficos), el modelo puede tener mejor rendimiento para ciertos estudiantes que otros.

Uso responsable: El sistema debe ser usado solo para propósitos educativos o administrativos con una política clara de uso y acceso.

